# 201. BERT로 텍스트 분류 - 한글

이 노트북에는 일반 텍스트 Naver 영화 리뷰 데이터 세트에 대한 감정 분석을 수행하기 위해 텍스트를 적절한 형식으로 사전 처리하는 방법과 BERT를 Fine Tuning하는 완전한 코드가 포함되어 있습니다.  

수행할 작업은 다음과 같습니다.

1. 데이터셋 불러오기
2. TensorFlow Hub에서 BERT 모델 로드
3. BERT와 분류기를 결합하여 나만의 모델 구축
4. BERT를 미세 조정하여 자신만의 모델을 훈련
5. 모델을 저장하고 문장 분류에 사용

In [ ]:
# A dependency of the preprocessing for BERT inputs

## Sentiment analysis

리뷰 텍스트를 기반으로 영화 리뷰를 *긍정* 또는 *부정*으로 분류합니다.

### naver movie dataset Download


train 시간 단축을 위해 데이터의 1/10 만 사용

In [ ]:
# AUTOTUNE을 사용하여 GPU 성능 자동 최적화
# 훈련 데이터셋 생성
# 배치 처리, 셔플, 캐시 및 프리페치 적용
# 테스트 데이터셋 생성
# 배치 처리, 셔플, 캐시 및 프리페치 적용

In [ ]:
# 훈련 데이터셋에서 한 배치 가져오기
    # 배치에서 처음 6개의 영화평과 레이블 출력
        # 영화평 출력 (바이트 문자열을 UTF-8로 디코딩)
        # 레이블 출력

## TensorFlow Hub에서 모델 로드

TensorFlow Hub에서 로드하고 미세 조정할 여러 BERT 모델을 선택할 수 있습니다.

  - [BERT-Base](https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3), [Uncased](https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/ 3) 원래 BERT 작성자가 발표한 가중치
  - [Small BERTs](https://tfhub.dev/google/collections/bert/1): 동일한 일반 아키텍처를 갖지만 더 적거나 더 작은 Transformer 블록을 사용하므로 속도, 크기 및 품질 간의 균형을 모색.
  - [ALBERT](https://tfhub.dev/google/collections/albert/1): 레이어 간에 매개변수를 공유하여 모델 크기(하지만 계산 시간은 아님)를 줄이는 4가지 크기의 "A Lite BERT".
  - [BERT Experts](https://tfhub.dev/google/collections/experts/bert/1): 모두 BERT 기반 아키텍처를 가지고 있지만 모델 목표 task 에 더 밀접하게 정렬하기 위해 8가지  서로 다른 사전 교육 도메인의 data 로 pre-train.
  - [Electra](https://tfhub.dev/google/collections/electra/1)는 BERT와 동일한 아키텍처(세 가지 다른 크기)를 갖지만 유사한 설정에서 생성적 적대 네트워크(GAN) 비슷한 discriminator로 사전 훈련.
  - Talking-Heads Attention 및 Gated GELU가 있는 BERT [[base](https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1), [large](https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_large/1)]는 Transformer 아키텍처의 핵심에 대한 두 가지 개선 사항을 가집니다.

미세 조정 속도를 빠르게 하기위해 Small BERT(더 적은 수의 매개변수 사용)로 시작합 니다. 작지만 더 높은 정확도를 원한다면 ALBERT가 다음 옵션이 될 수 있습니다. 더 나은 정확도를 원하면 클래식 BERT 크기 중 하나를 선택하거나 Electra, Talking Heads 또는 BERT Expert와 같은 최근 개선 사항 중 하나를 선택합니다.

아래에서 사용 가능한 모델 외에도 더 크고 더 나은 정확도를 얻을 수 있는 모델의 [여러 버전](https://tfhub.dev/google/collections/transformer_encoders_text/1)이 있지만  단일 GPU에서 fine-tuning 하기에 너무 큽니다. [TPU colab에서 BERT를 사용하여 GLUE 작업 해결](https://www.tensorflow.org/text/tutorials/bert_glue)에서 이 작업을 수행할 수 있습니다.

In [ ]:
#@title Choose a BERT model to fine-tune

## The preprocessing model

- 텍스트 입력은 BERT에 입력되기 전에 숫자 토큰 ID로 변환되고 여러 Tensor에 정렬되어야 합니다.   

- TensorFlow Hub는 위에서 설명한 각 BERT 모델에 대해 일치하는 전처리 모델을 제공하며, 이는 TF.text 라이브러리의(tensorflow_text) TF 작업을 사용하여 이 변환을 구현합니다. 텍스트를 사전 처리하기 위해 TensorFlow 모델 외부에서 순수 Python 코드를 실행할 필요는 없습니다.

- 전처리 모델은 위에 인쇄된 URL에서 읽을 수 있는 BERT 모델 문서에서 참조하는 모델이어야 합니다. 위의 드롭다운에서 BERT 모델의 경우 전처리 모델이 자동으로 선택됩니다.

hub.KerasLayer()는 tensorflow hub에 저장된 모델을 불러올 수 있습니다.
```
seq_length = 128  # Your choice here.
bert_preprocess_model = hub.KerasLayer(
    tfhub_handle_preprocess,
    arguments=dict(seq_length=seq_length))  # Optional argument.
```

BERT 모델이 사용할 사전 처리의 3가지 출력(`input_words_id`, `input_mask` 및 `input_type_ids`)이 있습니다.

- `input_word_ids` : 토큰 인덱스, 모델에서 입력으로 사용할 시퀀스를 구축하는 토큰의 숫자 표현. 여기서는 128 개 token 으로 길이가 제한됩니다.

- `input_type_ids`: 한 쌍의 문장 또는 질문 답변에 대한 분류 시 사용 .영화 평은 단일 문장 입력이기 때문에 0 만 갖습니다. 두개의 문장 입력의 경우 각 입력에 대해 0, 1 로 구분됩니다.
```
[CLS] SEQUENCE_A [SEP] SEQUENCE_B [SEP]
ex) [CLS] HuggingFace is based in NYC [SEP] Where is HuggingFace based? [SEP]
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
```

- `input_mask` : `1`은 입력 id 값을 나타내고 `0`은 패딩된 값을 나타냅니다.  

이 텍스트 전처리기는 TensorFlow 모델이므로 모델에 직접 포함될 수 있습니다.

In [ ]:
# 전처리 모델 불러오기

일부 텍스트에 대한 전처리 모델을 시도하고 출력을 살펴보겠습니다.

## BERT 모델 사용

BERT를 자신의 모델에 적용하기 전에 출력을 살펴보겠습니다. TF Hub에서 로드하고 반환된 값을 확인합니다.

<img src="https://editor.analyticsvidhya.com/uploads/84063bert_em.png" width=600 />

BERT 모델은 `pooled_output`, `sequence_output`, `encoder_outputs`의 3가지 중요한 키가 있는 맵을 반환합니다.

- `pooled_output`은 각 입력 시퀀스를 전체적으로 나타냅니다. shape은 `[batch_size, H]`입니다. 이것은 **전체 영화 리뷰에 대한 임베딩**으로 생각할 수 있습니다.
- `sequence_output`은 컨텍스트의 각 입력 토큰을 나타냅니다. shape은 `[batch_size, seq_length, H]`입니다. 이것을 영화 리뷰의 모든 토큰에 대한 컨텍스트 임베딩으로 생각할 수 있습니다.
- `encoder_outputs`는 'L'개의 Transformer 블록의 중간 activation입니다. `outputs["encoder_outputs"][i]`는 `0 <= i < L`에 대해 `i`번째 Transformer 블록의 출력이 있는 `[batch_size, seq_length, H]` 모양의 텐서입니다. list의 마지막 값은 `sequence_output`과 같습니다.

우리의 fine-tuning을 위해서는 `pooled_output` array를 사용합니다.

In [ ]:
# encoder 모델 불러오기

## 모델 정의

전처리 모델, 선택된 BERT 모델과 하나의 Dense 및 Dropout 레이어를 사용하여 매우 간단한 fine-tuning된 모델을 생성합니다.


In [ ]:
def build_classifier_model():
    # 텍스트 입력 레이어 정의 (형태: 문자열, 이름: 'text')
    # 전처리 모델 (허브에서 제공하는 전처리 레이어)
    # 사전 학습된 BERT 모델 (허브에서 제공하는 인코더 레이어, 학습 가능)
    # 마지막 레이어 추가 (Pooled output 사용)
    # 모델 생성 및 반환

모델이 전처리 모델의 출력으로 실행되는지 확인합니다.

In [ ]:
# 분류 모델 생성
# 테스트 텍스트 데이터로 모델 예측 수행
# 예측 결과 출력
# 시그모이드 함수를 적용한 결과 출력

물론 모델이 아직 훈련되지 않았기 때문에 출력은 의미가 없습니다. 모델의 구조를 살펴봅니다.

## Model training

### Loss function

이것은 이진 분류 문제이고 모델이 확률을 출력하므로 'losses.BinaryCrossentropy' 손실 함수를 사용합니다.


### Optimizer

미세 조정을 위해 BERT가 원래 훈련된 것과 동일한 최적화 프로그램인 "Adaptive Moments"(Adam)를 사용하겠습니다. 이 옵티마이저는 예측 손실을 최소화하고 [AdamW](https://arxiv.org/abs/1711.05101)라고도 하는 가중치 감쇠(모멘트를 사용하지 않음)로 정규화합니다.

학습률(`init_lr`)의 경우 BERT 사전 학습과 동일한 schedule을 사용합니다.

In [ ]:
# 에포크 수 설정
# 에포크당 스텝 수 계산
# 전체 학습 스텝 수 계산
# 워밍업 스텝 수 계산 (전체 학습 스텝 수의 10%)
# 초기 학습률 설정

훈련 시간은 선택한 BERT 모델의 복잡성에 따라 달라집니다. 작업시간 T4 GPU 기준 약 6시간 소요.

### Evaluate the model


### Plot the accuracy and loss over time

## Export for inference

### Fine-tuning 한 모델 저장

### 저장한 모델 reload

원하는 문장으로 모델을 테스트할 수 있습니다. 아래의 예제 변수에 추가하기만 하면 됩니다.

In [ ]:
def print_my_examples(inputs, results):

## Next steps

Naver movie review의 29만개 전체 dataset 사용하여 Fine Tuning